In [48]:
import netCDF4
import os
import glob
import json

In [49]:
moi_files = glob.glob('/storage/data/hpc/mnt/moi/*')
qmap = {
    'sic4dvar':'Q_da',
    'sad': 'Qa',
    'hivdi':'reach/Q',
    'momma':'Q',
    'geobam':'q/q',
    'neobam':'q/q',
    'metroman':'allq'}

In [50]:
out_dict = {}

for moi_fp in moi_files[:40]:
    moi_data = netCDF4.Dataset(moi_fp)
    rid = os.path.basename(moi_fp).split("_")[0]
    obs_fp  = f'/storage/data/hpc/mnt/input/swot/{rid}_SWOT.nc'
    
    # print(moi_data)
    for moi_group in moi_data.groups:
        try:
            q = moi_data[moi_group]['q'][:]

            if any(q):
                out_dict[rid] = {}
                out_dict[rid][moi_group] = {}
                obs_fp  = f'/storage/data/hpc/mnt/input/swot/{os.path.basename(moi_fp).split("_")[0]}_SWOT.nc'
                obs_data = netCDF4.Dataset(obs_fp)
                time = obs_data['reach']['time_str'][:]
                all_decoded_times = []
                for a_time in time:
                    try:
                        string = ''.join(byte.decode('utf-8') for byte in a_time)
                        all_decoded_times.append(string)
                    except:
                        all_decoded_times.append('nan_time')
                for index, one_q in enumerate(q):
                    if one_q > 0:
                        out_dict[rid][moi_group][all_decoded_times[index]] = one_q

                
                obs_data.close()
        except Exception as e:
            print(e)
            # print(moi_group, 'fail')
            pass

    moi_data.close()
with open('discharge_analysis.json', 'w') as f:
    json.dump(out_dict, f)
